In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import pandas as pd
import os
#import pickle
from utility import *
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV

In [2]:
print(tf.__version__)

1.4.0


In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

def mlp_model_fn(features, labels, mode, params):
  """Model function for MLP."""
  
  #TODO: need works on this
  config = params


  # Input Layer
  
  input_layer = tf.reshape( features["x"], [-1, features["x"].shape[1] ] )
  #print ('feature x', features["x"])
  #print ('feature x shape', features["x"].shape)
  #print ('reshape:', input_layer)
  #print ('reshape shape:', input_layer.shape)
  #trans = tf.string_to_number(input_layer)
  #print ('trans reshape:', trans)
  #print ('reshape shape:', input_layer.shape)


  # Dense Layers
  hidden1 = tf.layers.dense(inputs=input_layer, units=config['n_hidden1'], activation=tf.nn.relu)
  normal_1 = tf.layers.batch_normalization(hidden1, training=mode == tf.estimator.ModeKeys.TRAIN)
 
  drop_h1 = tf.layers.dropout(
      inputs=normal_1, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  hidden2 = tf.layers.dense(inputs=drop_h1, units=config['n_hidden2'], activation=tf.nn.relu)
  drop_h2 = tf.layers.dropout(
      inputs=hidden2, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  if config['n_hidden3'] != None:
    hidden3 = tf.layers.dense(inputs=drop_h2, units=config['n_hidden3'], activation=tf.nn.relu)
    drop_h3 = tf.layers.dropout(
        inputs=hidden3, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=drop_h3, units=config['nclasses'])
  else:
    logits = tf.layers.dense(inputs=drop_h2, units=config['nclasses'])
    
  predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  # obsolete logits = tf.Print(logits,predictions["probabilities"])
  

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=config['nclasses'])
  #print (onehot_labels)
  #print (logits)
  loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)


  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)

    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  print ('prob shape',predictions["probabilities"].shape)
  print ('prob ',predictions["probabilities"])
  print ('prob ',predictions["probabilities"])
  k_3 = tf.nn.in_top_k(predictions=predictions["probabilities"], targets= labels,k = 3)
  k_3 = tf.cast(k_3, tf.float32)
  print (k_3)
  eval_metric_ops = {
      #"accuracy": tf.metrics.accuracy(
      #    labels=labels, predictions=predictions["classes"]),
      "top-3 accuracy": tf.metrics.average_precision_at_k(
                        labels = labels,
                        predictions = predictions["probabilities"],
                        k = 3)
          
      #"top-5 accuracy": tf.metrics.mean(
      #   tf.nn.in_top_k(predictions=predictions["probabilities"], 
      #                   targets= labels,k = 5)) 
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [ ]:
unused_argv = ['pilot_mlp_newAPI.py', 'data_1p_companded.csv', 'model/', 1024, 512, 256]

filename = unused_argv[1]

'''
if os.path.isfile('ft_to_idx.npy') and os.path.isfile('idx_to_ft.npy') and os.path.isfile('nclasses.npy') and os.path.isfile('group_data.npy'):
    ft_to_idx = np.load('ft_to_idx.npy')
    ft_to_idx = ft_to_idx.item()
    idx_to_ft = np.load('idx_to_ft.npy')
    idx_to_ft = idx_to_ft.item()
    nclasses = np.load('nclasses.npy')
    #f = open('group_data.pkl','r')
    #group_data = pickle.load('group_data.pkl')
    group_data = np.load('group_data.npy') 
    #group_data = np.load('toy_data.npy') # for proof of algo purpose, real use case should use the above line
    group_data = group_data.item()

else:
    ft_to_idx, idx_to_ft, nclasses, group_data = prepare_file(filename)
    np.save("ft_to_idx", ft_to_idx)
    np.save("idx_to_ft", idx_to_ft)
    np.save("nclasses", nclasses)
    #f = open('group_data.pkl','w')
    #pickle.dump(group_data, f)
    np.save("group_data", group_data)

#gen train set
cnt = map(lambda x: len(x),group_data.values())
percentile_40 = np.percentile(cnt, 40)
train, dev ,test = train_dev_test_split(group_data, proportion = [0.6,0.2], thre = percentile_40) # should be 1000 or so
train_data, train_labels = gen_feed(train, ft_to_idx, upper_limit=12000)
del train, group_data
dev_data, dev_labels = gen_feed(dev, ft_to_idx, upper_limit=3000)
del dev
test_data, test_labels = gen_feed(test, ft_to_idx, upper_limit=3000)
del test
train_data = train_data.astype(np.float32) 
dev_data = dev_data.astype(np.float32)
test_data = test_data.astype(np.float32) 
'''
ft_to_idx = np.load('ft_to_idx.npy')
ft_to_idx = ft_to_idx.item()
idx_to_ft = np.load('idx_to_ft.npy')
idx_to_ft = idx_to_ft.item()

dev_data = np.load("dev_data.npy")
dev_labels = np.load("dev_labels.npy")
test_data = np.load("test_data.npy")
test_labels = np.load("test_labels.npy")
train_data = np.load("train_data.npy")
train_labels = np.load("train_labels.npy")




In [ ]:
scaler = StandardScaler()
train_data_norm = scaler.fit_transform(train_data)
dev_data_norm = scaler.transform(dev_data)
test_data = scaler.transform(test_data)

In [ ]:
from itertools import product
hidden_units = [[1024,1024],[1024,512,256],[256,128,100]]#,[2048,2048],[2048,1024,512]]
batch_sizes = [64,128]#,256,512]
dropouts = [0.4,0.2]

#reset config, defined in previous block
config = {}
config['nclasses'] = len(np.unique(train_labels))
results = []
for hidden, batch_size, dropout in product(hidden_units,batch_sizes,dropouts):
    config['n_hidden1'] = hidden[0]
    config['n_hidden2'] = hidden[1]
    try: config['n_hidden3'] = hidden[2]
    except: config['n_hidden3'] = None
    config['dropout'] = dropout
    # Create the Estimator
    mlp_classifier = tf.estimator.Estimator(
    model_fn=mlp_model_fn, model_dir=None, params=config)

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=5000)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=batch_size,
      num_epochs=3,
      shuffle=True)

    steps = train_data.shape[0]/batch_size
    #print (steps)
    mlp_classifier.train(
      input_fn=train_input_fn,
      steps=None, # 60000
      hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": dev_data},
        y=dev_labels,
        num_epochs=1,
        shuffle=False)

    eval_results = mlp_classifier.evaluate(input_fn=eval_input_fn)
    print ('----setting----')
    print ('hidden units:',hidden)
    print ('batch_size:', batch_size)
    print ('dropout:', dropout)
    print ('----performance----')
    print(eval_results)
    results.append((eval_results['accuracy'], hidden, batch_size, dropout))

In [ ]:
results = sorted(results, key=lambda x: x[0], reverse=True)
print (results)

In [ ]:
best_param = results[0]
hidden = best_param[1]
config['n_hidden1'] = hidden[0]
config['n_hidden2'] = hidden[1]
try: config['n_hidden3'] = hidden[2]
except: config['n_hidden3'] = None
batch_size = best_param[2]
config['dropout'] = best_param[3]

# Create the Estimator
cnn_classifier = tf.estimator.Estimator(
model_fn=mlp_model_fn, model_dir=None, params=config)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=5000) 

#make full training data
train_full_data = np.vstack((train_data,dev_data))
train_full_labels = np.hstack((train_labels,dev_labels))
train_full_data = scaler.fit_transform(train_full_data)
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_full_data},
  y=train_full_labels,
  batch_size=batch_size,
  num_epochs=3,
  shuffle=True)

steps = train_full_data.shape[0]/batch_size
cnn_classifier.train(
  input_fn=train_input_fn,
  steps=None, # 60000
  hooks=[logging_hook])

pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    num_epochs=1,
    shuffle=False)
predictions = mlp_classifier.predict(input_fn=pred_input_fn)
print (list(p["probabilities"] for p in predictions))
predictions = list(p["classes"] for p in predictions)

print ('----Best setting----')
print ('hidden units:',hidden)
print ('batch_size:', batch_size)
print ('dropout:', dropout)
print ('\n')

target_names = np.unique(test_labels)
target_names = [idx_to_ft[i] for i in target_names]
print (classification_report(test_labels, predictions, target_names=target_names))

In [ ]:
print (classification_report(test_labels, predictions))

In [ ]:
from sklearn.metrics import accuracy_score
print (accuracy_score(test_labels, predictions))

In [ ]:
uniq = np.unique(train_full_labels)

In [ ]:
for i , lab in enumerate(uniq):
    print (i, target_names[i])

In [ ]:
np.unique(train_labels) == np.unique(test_labels)